In [1]:
import pandas as pd

report_list = pd.read_csv('../data/preprocessed/20210104_D002_list.csv',
                          encoding='cp949',
                          dtype=object
                          )
report_list.shape

(112, 9)

In [2]:
report_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   corp_code   112 non-null    object
 1   corp_name   112 non-null    object
 2   stock_code  112 non-null    object
 3   corp_cls    112 non-null    object
 4   report_nm   112 non-null    object
 5   rcept_no    112 non-null    object
 6   flr_nm      112 non-null    object
 7   rcept_dt    112 non-null    object
 8   rm          1 non-null      object
dtypes: object(9)
memory usage: 8.0+ KB


In [3]:
report_list.head()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,01262023,루트락,253610,N,임원ㆍ주요주주특정증권등소유상황보고서,20210104000506,김성구,20210104,NaN
1,00291860,조광ILI,044060,K,임원ㆍ주요주주특정증권등소유상황보고서,20210104000504,이브이첨단소재,20210104,NaN
2,00927558,유바이오로직스,206650,K,[기재정정]임원ㆍ주요주주특정증권등소유상황보고서,20210104000502,전동운,20210104,NaN
3,01262023,루트락,253610,N,임원ㆍ주요주주특정증권등소유상황보고서,20210104000497,엘앤케이바이오,20210104,NaN
4,00190321,케이티,030200,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210104000496,김봉기,20210104,NaN


### Download data

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get('DART_API_KEY')

In [40]:
from requests import Session
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

session = Session()
retry = Retry(connect=5, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('https://', adapter)
session.mount('http://', adapter)

URL = 'https://opendart.fss.or.kr/api/document.xml'

FILEPATH = '../data/origin/dart/reports/d002/'

RAW_FILENAME = '{rcept_no}.{extension}'
MY_FILENAME = '{corp_name}_d002_{rcept_no}.{extension}'

In [19]:
def download_report(corp_name:str, rcept_no: str) -> bool:
    # try:
        params = {
            'crtfc_key': API_KEY, 
            'rcept_no': rcept_no
            }
        
        res = session.get(URL, params=params)
        
        filepath = FILEPATH + MY_FILENAME.format(corp_name=corp_name, rcept_no=rcept_no, extension='xml')
        with open(filepath, 'wb') as f:
            f.write(res.content)
        
        return True
    # except:
    #     return False

In [20]:
for i, report in report_list.iterrows():
    corp_name = report['corp_name']
    rcept_no = report['rcept_no']
    download_report(corp_name, rcept_no)

## Extract data

In [57]:
import os
import re
import zipfile
from typing import *

MY_PATTERN = r'([^_]+)_d002_(\d+).zip'

def get_name_and_report_code(filename: str) -> Tuple[str, str]:
    name, report_code = re.search(MY_PATTERN, filename).groups()
    return name, report_code
    
def extract_report(name: str, report_code: str)-> Tuple[str, str]:
    zipfile.ZipFile(FILEPATH + MY_FILENAME.format(corp_name=name, rcept_no=report_code, extension='zip'))\
        .extractall(FILEPATH)
    
    return name, report_code

def rename_report(name: str, report_code: str):
    os.rename(FILEPATH + RAW_FILENAME.format(rcept_no=report_code, extension='xml'),
              FILEPATH + MY_FILENAME.format(corp_name=name, rcept_no=report_code, extension='xml'))

In [58]:
filenames = [filename for filename in os.listdir(FILEPATH) if filename[-4:] == '.zip']
error = []

for filename in filenames:
    try:
        name, report_code = get_name_and_report_code(filename)
        extract_report(name, report_code)
        rename_report(name, report_code)
    except:
        error.append((name, report_code))
    # break

NameError: name 'bs' is not defined